In [2]:
!pip install -Uq openai-agents

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.6/130.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.9/155.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.8 MB/s eta 0:00:00


In [3]:
import nest_asyncio
nest_asyncio.apply()

In [4]:
from google.colab import userdata

gemini_api_key = userdata.get("GEMINI_API_KEY")
if gemini_api_key:
  print("Yes")
else:
  print("No")

Yes


In [5]:
from agents import Agent, Runner, OpenAIChatCompletionsModel
from openai import AsyncOpenAI

client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

model = OpenAIChatCompletionsModel(
    openai_client = client,
    model = "gemini-2.0-flash"
)


In [10]:
import asyncio
from pydantic import BaseModel, Field
from agents import Agent, Runner

# ------- Structured Output Model --------
class WorkOutPlan(BaseModel):
  """Workout recommendations with exercises and details."""
  focus_area : str = Field(description="Primary focus of the workout.")
  difficulty : str = Field(description="Workout difficulty level. (Easy, Medium, Hard)")
  exercises : list[str] = Field(description="List of recommended exercises")
  notes : str = Field(description="Additional notes or recommendations")


In [11]:
# -------- Simple Fitness Agent ---------

fitness_agent = Agent(
    name = "Fitness Agent",
    instructions = """You are a trained and one of the best in town fitness agent, you need to recommend the client with exercises according to the areas of focus.
    When asked about suggestions, you need to:
    - Determine their fitness goal (weight loss, muscle gain, endurance)
    - Keep in mind if the have any medical condition
    - Create an appropriate workout plan.
    - Included safety tips and notes.
    """,
    model = model,
    output_type = WorkOutPlan
)

In [17]:
async def main():
  input = "I want to loose fat around my abdomen, suggest me a diet plan and exercies. I have no medical conditions."

  print("\n", "*"*50)
  print(f"QUERY:  {input}")
  print("="*50)

  whatever = await Runner.run(fitness_agent, input)
  print("STRUCTURED RESPONSE")
  print(whatever)
  print(whatever.final_output)
  print(whatever.final_output.dict)
  print(whatever.final_output.notes)

if__name__="__main__"
asyncio.run(main())


 **************************************************
QUERY:  I want to loose fat around my abdomen, suggest me a diet plan and exercies. I have no medical conditions.


STRUCTURED RESPONSE
RunResult:
- Last agent: Agent(name="Fitness Agent", ...)
- Final output (WorkOutPlan):
    {
      "focus_area": "Abdominal Fat Loss",
      "difficulty": "Medium",
      "exercises": [
        "Crunches (3 sets of 15-20 reps)",
        "Leg Raises (3 sets of 15-20 reps)",
        "Russian Twists (3 sets of 15-20 reps)",
        "Plank (3 sets, hold for 30-60 seconds)",
        "Bicycle Crunches (3 sets of 15-20 reps)",
        "Cardio: Running (30-45 minutes, 3-4 times per week)",
        "Cardio: Swimming (30-45 minutes, 3-4 times per week)",
        "High-Intensity Interval Training (HIIT) (20-30 minutes, 2-3 times per week)"
      ],
      "notes": "Combine these exercises with a balanced diet focused on whole foods, lean proteins, healthy fats, and complex carbohydrates. Drink plenty of water and aim for a calorie deficit to promote fat loss. Consistency is key. Listen to your body and adjust the intensity as needed."
    }
- 1 new item(s)
- 1 raw response(s)
